In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np

%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
x_init = np.random.randn(2,6).astype(np.float32)
x2_init = np.random.randn(3).astype(np.float32)
U_init = np.random.randn(3,3).astype(np.float32)
V_init = np.random.randn(3,3).astype(np.float32)
W_init = np.random.randn(6,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

x = DenseTensor(x_init)
W = DenseTensor(W_init)
m = DenseTensor(m_init)
out = x.dot(W).relu()
out = out.logsoftmax()
out = out.mul(m).add(m).sum()
out.backward()

out.cpu().data, x

x2 = DenseTensor(x2_init)#.gpu()
W = SparseTensor(W_init)
out = W.dot(x2).relu().sum()

out.backward()

out.cpu().data, x

In [3]:
import numpy as np
import pyopencl as cl

mf = cl.mem_flags

In [4]:
dim1 = 16
dim2 = 32
dim3 = 24
bs = dim3

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.4

a = np.zeros((dim1,dim2))
b = np.zeros((dim2,dim3)).astype(np.float32)

a.shape, b.shape

((16, 32), (32, 24))

In [5]:
def fill_sparse(mat, sparsity=0.1):
    indices = np.array(range(mat.shape[1]))
    nrows = int(mat.shape[1]*sparsity)
    for row in range(mat.shape[0]):
        lim = nrows #+ int(np.random.random()*3)
        mat[row][np.random.permutation(indices)[:lim]] = np.random.random(lim)
    return mat

a = fill_sparse(a, sparsity).astype(np.float32)
b = fill_sparse(b, sparsity).astype(np.float32)

In [6]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [7]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [8]:
x2_init.T

array([-0.5651575 , -0.72725755,  0.7735674 ], dtype=float32)

In [9]:
mult = a.dot(b)
mult.shape

(16, 24)

In [10]:
mult.shape

(16, 24)

In [11]:
def to_data(mat):
    all_rows = []
    all_idxs = []
    all_nnzs = []
    for row in range(mat.shape[0]):
        rowdata = []
        colidxs = []
        all_nnzs.append(0)
        for col in range(mat.shape[1]):
            val = mat[row][col]
            if val != 0:
                rowdata.append(val)
                colidxs.append(col)
                all_nnzs[-1] += 1
        all_rows.append(rowdata)
        all_idxs.append(colidxs)
    
    ellwidth = min(int(np.sqrt(np.max(all_nnzs))+1)**2, mat.shape[1])
    #all_rows = np.array(all_rows)#.astype(np.float32).flatten()           
    for row in range(mat.shape[0]):
        #print(row, all_rows)
        all_rows[row] = np.array(all_rows[row])
        all_rows[row].resize(ellwidth)
        all_idxs[row] = np.array(all_idxs[row])
        all_idxs[row].resize(ellwidth)
        #print(all_idxs[row])
    all_rows = np.array(all_rows)
    all_idxs = np.array(all_idxs)
    all_nnzs = np.array(all_nnzs)
    
#     while (not all_rows[:,-1].any()):
#         all_rows = all_rows[:,:-1]
#         all_idxs = all_idxs[:,:-1]
#         ellwidth -= 1
        
    
    all_rows = np.array(all_rows).astype(np.float32).flatten()
    all_idxs = np.array(all_idxs).astype(np.uint32).flatten()
    
    all_nnzs = np.array(all_nnzs).astype(np.uint32)
    
    
    return all_rows, all_idxs, all_nnzs, ellwidth

In [12]:
def to_dense(data, cols, nnzs, ellw, shape):
    out = np.zeros(shape)
    for row in range(shape[0]):
        for icol in range(nnzs[row]):
            out[row,cols[row*ellw+icol]] = data[row*ellw+icol]
    return out

In [13]:
adata, acols, annz, ellwa = to_data(a)
adata, acols, annz, ellwa

(array([0.16797899, 0.87880903, 0.4000462 , 0.60551333, 0.3397555 ,
        0.12211707, 0.61375475, 0.8297732 , 0.25965056, 0.85697263,
        0.7936516 , 0.13828662, 0.        , 0.        , 0.        ,
        0.        , 0.89125365, 0.93762594, 0.03801361, 0.13324429,
        0.35636038, 0.4082106 , 0.9805923 , 0.6787518 , 0.478064  ,
        0.447243  , 0.06849757, 0.82752186, 0.        , 0.        ,
        0.        , 0.        , 0.16898978, 0.27443814, 0.9523248 ,
        0.35010964, 0.12153614, 0.5153773 , 0.30960494, 0.48031798,
        0.959742  , 0.40850204, 0.47222662, 0.24772032, 0.        ,
        0.        , 0.        , 0.        , 0.46642095, 0.46711934,
        0.8655582 , 0.40356433, 0.759128  , 0.48451155, 0.3316039 ,
        0.740936  , 0.51790565, 0.758095  , 0.52187777, 0.61482006,
        0.        , 0.        , 0.        , 0.        , 0.7123387 ,
        0.33271232, 0.31538633, 0.84150213, 0.9183877 , 0.40973175,
        0.01835028, 0.33034727, 0.298536  , 0.37

In [14]:
adatat, acolst, annzt, ellwat = to_data(a.T)
adatat, acolst, annzt, ellwat

(array([0.16898978, 0.7123387 , 0.5910008 , 0.06666391, 0.4507213 ,
        0.41975817, 0.5913447 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.89125365, 0.03724516, 0.42652833, 0.74566156,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.93762594, 0.10916828, 0.49283606,
        0.8300311 , 0.11515573, 0.9487138 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.16797899, 0.27443814,
        0.7437254 , 0.6381501 , 0.6501674 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.03801361,
        0.9523248 , 0.46642095, 0.80913323, 0.504041  , 0.41625652,
        0.05185964, 0.7145313 , 0.        , 0.  

In [15]:
bdata, bcols, bnnz, ellwb = to_data(b)
bdata, bcols, bnnz, ellwb

(array([0.2106778 , 0.6727729 , 0.994735  , 0.7231555 , 0.8717573 ,
        0.00371827, 0.47682822, 0.12149344, 0.7363708 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.1913773 , 0.34261224, 0.26097038, 0.9146003 ,
        0.96853554, 0.3060288 , 0.10322573, 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.76457417, 0.5253448 , 0.58099407,
        0.48766515, 0.143648  , 0.80803716, 0.8913566 , 0.9567756 ,
        0.45077503, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.26847026, 0.4472989 ,
        0.45999217, 0.2888692 , 0.3502671 , 0.5811538 , 0.29828227,
        0.1188754 , 0.5012796 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.5153698 ,
        0.71913767, 0.500461  , 0.17228748, 0.00423898, 0.88242525,
        0.68886006, 0.58187085, 0.3876178 , 0.  

In [16]:
bdatat, bcolst, bnnzt, ellwbt = to_data(b.T)
adatat, bcolst, bnnzt, ellwbt

(array([0.16898978, 0.7123387 , 0.5910008 , 0.06666391, 0.4507213 ,
        0.41975817, 0.5913447 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.89125365, 0.03724516, 0.42652833, 0.74566156,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.93762594, 0.10916828, 0.49283606,
        0.8300311 , 0.11515573, 0.9487138 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.16797899, 0.27443814,
        0.7437254 , 0.6381501 , 0.6501674 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.03801361,
        0.9523248 , 0.46642095, 0.80913323, 0.504041  , 0.41625652,
        0.05185964, 0.7145313 , 0.        , 0.  

In [17]:
adense = to_dense(adata, acols, annz, ellwa, a.shape)

In [18]:
adenset = to_dense(adatat, acolst, annzt, ellwat, a.T.shape)

In [19]:
bdense = to_dense(bdata, bcols, bnnz, ellwb, b.shape)

In [20]:
bdenset = to_dense(bdatat, bcolst, bnnzt, ellwbt, b.T.shape)

In [21]:
adense

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.33975551,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.82977319, 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.79365158, 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.40821061, 0.        ,
        0.        , 0.        , 0.98059231, 0.        , 0.        ,
        0.        , 0.67875183, 0.478064  , 0.        , 0.        ,
        0.44724301, 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.95232481,
        0.35010964, 0.        , 0.        , 0.1215

In [22]:
adenset.T == adense

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  

In [23]:
bdenset.T == bdense

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,

In [24]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [25]:
a == adense

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  

In [26]:
a.shape

(16, 32)

In [27]:
adata.shape, acols.shape, annz.shape, ellwa

((256,), (256,), (16,), 16)

In [28]:
#acols = acols.astype(np.uint32)
#annz = annz.astype(np.uint32)

In [29]:
adata, acols, annz, b

(array([0.16797899, 0.87880903, 0.4000462 , 0.60551333, 0.3397555 ,
        0.12211707, 0.61375475, 0.8297732 , 0.25965056, 0.85697263,
        0.7936516 , 0.13828662, 0.        , 0.        , 0.        ,
        0.        , 0.89125365, 0.93762594, 0.03801361, 0.13324429,
        0.35636038, 0.4082106 , 0.9805923 , 0.6787518 , 0.478064  ,
        0.447243  , 0.06849757, 0.82752186, 0.        , 0.        ,
        0.        , 0.        , 0.16898978, 0.27443814, 0.9523248 ,
        0.35010964, 0.12153614, 0.5153773 , 0.30960494, 0.48031798,
        0.959742  , 0.40850204, 0.47222662, 0.24772032, 0.        ,
        0.        , 0.        , 0.        , 0.46642095, 0.46711934,
        0.8655582 , 0.40356433, 0.759128  , 0.48451155, 0.3316039 ,
        0.740936  , 0.51790565, 0.758095  , 0.52187777, 0.61482006,
        0.        , 0.        , 0.        , 0.        , 0.7123387 ,
        0.33271232, 0.31538633, 0.84150213, 0.9183877 , 0.40973175,
        0.01835028, 0.33034727, 0.298536  , 0.37

## MatMul (Sparse-Dense)

adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      float sum = 0;
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[col*ncols+gid2];
          //if (gid==0 && gid2==2)
          //  printf("aval, xval: %.2f,%.2f: (%i,%i) \\n", aval, xval, col, index);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [30]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          uint xidx = col*ncols+gid2;
          float xval  = vector_x[xidx];
          if (gid==0 && gid2==1)
            printf("aval, xval: %.2f,%.2f: (%i,%i) - %i \\n", aval, xval, col, index, xidx);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [31]:
a.shape, b.shape

((16, 32), (32, 24))

In [32]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [33]:
rows = a.shape[0]

In [34]:
mult = mult.astype(np.float32)

In [35]:
outshape = (a.shape[0], b.shape[1])
outshape

(16, 24)

In [36]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul2  # Use this Kernel object for repeated calls
knl(queue, [outshape[0]], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), np.uint32(outshape[1]), b_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.17,0.45: (3,0) - 73 
aval, xval: 0.88,0.12: (8,1) - 193 
aval, xval: 0.40,0.00: (10,2) - 241 
aval, xval: 0.61,0.00: (12,3) - 289 
aval, xval: 0.34,0.00: (14,4) - 337 
aval, xval: 0.12,0.13: (18,5) - 433 
aval, xval: 0.61,0.00: (19,6) - 457 
aval, xval: 0.83,0.44: (20,7) - 481 
aval, xval: 0.26,0.49: (21,8) - 505 
aval, xval: 0.86,0.00: (26,9) - 625 
aval, xval: 0.79,0.54: (28,10) - 673 
aval, xval: 0.14,0.00: (30,11) - 721 


In [37]:
(res_np-mult).sum()

1.527369e-07

In [38]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [39]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [40]:
res_buf

In [41]:
res_np

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243394 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898017 , 2.1122162 , 1.2259226 , 2.4700487 , 0.12201902,
        0.72471607, 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.94957584, 1.1395756 , 1.061765  , 1.3816046 , 0.33119798,
        1.1913491 , 0.52872694, 0.81870705, 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [42]:
mult

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [43]:
res_np==mult

array([[ True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False,  True, False,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False,  True,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,

In [44]:
res_np.shape

(16, 24)

In [45]:
mult.shape

(16, 24)

## MatMul (dense * sparse)

In [46]:
bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdata)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
bdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdatat)
bcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcolst)
bnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnzt)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // DENSE x SPARSE
    __kernel void matmul(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            uint   ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        uint nnz = rowNnz[gid2];
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid2 * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[gid*mwidth+col];
          sum  += aval * xval;
          if (gid==0 && gid2==0)
            printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [47]:
a.shape, b.shape

((16, 32), (32, 24))

In [48]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [49]:
rows = a.shape[0]

In [50]:
mult = mult.astype(np.float32)

In [51]:
outshape = np.array([a.shape[0], b.shape[1]])
outshape

array([16, 24])

In [52]:
b.T

array([[0.        , 0.        , 0.76457417, 0.26847026, 0.        ,
        0.        , 0.5677988 , 0.        , 0.        , 0.8750546 ,
        0.9323805 , 0.        , 0.        , 0.        , 0.16622874,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.39303797, 0.77240014, 0.        , 0.        , 0.04915274,
        0.        , 0.        , 0.        , 0.        , 0.4213172 ,
        0.14627703, 0.        ],
       [0.        , 0.        , 0.5253448 , 0.4472989 , 0.5153698 ,
        0.        , 0.        , 0.        , 0.12423625, 0.        ,
        0.        , 0.        , 0.        , 0.55864817, 0.        ,
        0.        , 0.9435074 , 0.1667168 , 0.12566799, 0.        ,
        0.43565068, 0.4873105 , 0.717685  , 0.57791394, 0.        ,
        0.80733645, 0.        , 0.6129844 , 0.53634363, 0.13901941,
        0.        , 0.        ],
       [0.        , 0.        , 0.58099407, 0.        , 0.71913767,
        0.1559078 , 0.        , 0.48234788, 0.    

In [53]:
a.T

array([[0.        , 0.        , 0.16898978, 0.        , 0.7123387 ,
        0.5910008 , 0.        , 0.        , 0.        , 0.06666391,
        0.4507213 , 0.41975817, 0.        , 0.        , 0.        ,
        0.5913447 ],
       [0.        , 0.89125365, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03724516, 0.        ,
        0.42652833, 0.        , 0.74566156, 0.        , 0.        ,
        0.        ],
       [0.        , 0.93762594, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.10916828,
        0.49283606, 0.8300311 , 0.11515573, 0.9487138 , 0.        ,
        0.        ],
       [0.16797899, 0.        , 0.27443814, 0.        , 0.        ,
        0.7437254 , 0.6381501 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.6501674 ,
        0.        ],
       [0.        , 0.03801361, 0.9523248 , 0.46642095, 0.        ,
        0.        , 0.80913323, 

In [54]:
outshape.T

array([16, 24])

In [55]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul  # Use this Kernel object for repeated calls
knl(queue, [outshape.T[0]], None, bdatat_buf, bcolst_buf, bnnzst_buf, np.uint32(ellwbt), np.uint32(b.shape[0]), np.uint32(outshape.T[1]), a_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.76,0.00 - 0.00: (2,0) 
aval, xval: 0.27,0.17 - 0.05: (3,1) 
aval, xval: 0.57,0.00 - 0.05: (6,2) 
aval, xval: 0.88,0.00 - 0.05: (9,3) 
aval, xval: 0.93,0.40 - 0.42: (10,4) 
aval, xval: 0.17,0.34 - 0.47: (14,5) 
aval, xval: 0.39,0.83 - 0.80: (20,6) 
aval, xval: 0.77,0.26 - 1.00: (21,7) 
aval, xval: 0.05,0.00 - 1.00: (24,8) 
aval, xval: 0.42,0.00 - 1.00: (29,9) 
aval, xval: 0.15,0.14 - 1.02: (30,10) 


In [56]:
(res_np-mult).sum()

1.527369e-07

In [57]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [58]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [59]:
res_buf

In [60]:
res_np

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243394 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898017 , 2.1122162 , 1.2259226 , 2.4700487 , 0.12201902,
        0.72471607, 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.94957584, 1.1395756 , 1.061765  , 1.3816046 , 0.33119798,
        1.1913491 , 0.52872694, 0.81870705, 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [61]:
mult

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [62]:
res_np==mult

array([[ True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False,  True, False,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False,  True,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,

In [63]:
res_np-mult

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         1.1920929e-07,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.1920929e-07, -2.3841858e-07,  0.0000000e+00, -2.3841858e-07,
         0.0000000e+00,  5.9604645e-08,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-5.9604645e-08,  0.0000000e+00,  0.0000000e+00,  1.1920929e-07,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -5.960

In [64]:
res_np.shape

(16, 24)

In [65]:
mult.shape

(16, 24)

## MatMul (dense * sparse)

In [46]:
bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdata)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
bdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdatat)
bcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcolst)
bnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnzt)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // DENSE x SPARSE
    __kernel void matmul(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            uint   ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        uint nnz = rowNnz[gid2];
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid2 * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[gid*mwidth+col];
          sum  += aval * xval;
          if (gid==0 && gid2==0)
            printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [47]:
a.shape, b.shape

((16, 32), (32, 24))

In [48]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [49]:
rows = a.shape[0]

In [50]:
mult = mult.astype(np.float32)

In [51]:
outshape = np.array([a.shape[0], b.shape[1]])
outshape

array([16, 24])

In [52]:
b.T

array([[0.        , 0.        , 0.76457417, 0.26847026, 0.        ,
        0.        , 0.5677988 , 0.        , 0.        , 0.8750546 ,
        0.9323805 , 0.        , 0.        , 0.        , 0.16622874,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.39303797, 0.77240014, 0.        , 0.        , 0.04915274,
        0.        , 0.        , 0.        , 0.        , 0.4213172 ,
        0.14627703, 0.        ],
       [0.        , 0.        , 0.5253448 , 0.4472989 , 0.5153698 ,
        0.        , 0.        , 0.        , 0.12423625, 0.        ,
        0.        , 0.        , 0.        , 0.55864817, 0.        ,
        0.        , 0.9435074 , 0.1667168 , 0.12566799, 0.        ,
        0.43565068, 0.4873105 , 0.717685  , 0.57791394, 0.        ,
        0.80733645, 0.        , 0.6129844 , 0.53634363, 0.13901941,
        0.        , 0.        ],
       [0.        , 0.        , 0.58099407, 0.        , 0.71913767,
        0.1559078 , 0.        , 0.48234788, 0.    

In [53]:
a.T

array([[0.        , 0.        , 0.16898978, 0.        , 0.7123387 ,
        0.5910008 , 0.        , 0.        , 0.        , 0.06666391,
        0.4507213 , 0.41975817, 0.        , 0.        , 0.        ,
        0.5913447 ],
       [0.        , 0.89125365, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03724516, 0.        ,
        0.42652833, 0.        , 0.74566156, 0.        , 0.        ,
        0.        ],
       [0.        , 0.93762594, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.10916828,
        0.49283606, 0.8300311 , 0.11515573, 0.9487138 , 0.        ,
        0.        ],
       [0.16797899, 0.        , 0.27443814, 0.        , 0.        ,
        0.7437254 , 0.6381501 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.6501674 ,
        0.        ],
       [0.        , 0.03801361, 0.9523248 , 0.46642095, 0.        ,
        0.        , 0.80913323, 

In [54]:
outshape.T

array([16, 24])

In [55]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul  # Use this Kernel object for repeated calls
knl(queue, [outshape.T[0]], None, bdatat_buf, bcolst_buf, bnnzst_buf, np.uint32(ellwbt), np.uint32(b.shape[0]), np.uint32(outshape.T[1]), a_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.76,0.00 - 0.00: (2,0) 
aval, xval: 0.27,0.17 - 0.05: (3,1) 
aval, xval: 0.57,0.00 - 0.05: (6,2) 
aval, xval: 0.88,0.00 - 0.05: (9,3) 
aval, xval: 0.93,0.40 - 0.42: (10,4) 
aval, xval: 0.17,0.34 - 0.47: (14,5) 
aval, xval: 0.39,0.83 - 0.80: (20,6) 
aval, xval: 0.77,0.26 - 1.00: (21,7) 
aval, xval: 0.05,0.00 - 1.00: (24,8) 
aval, xval: 0.42,0.00 - 1.00: (29,9) 
aval, xval: 0.15,0.14 - 1.02: (30,10) 


In [56]:
(res_np-mult).sum()

1.527369e-07

In [57]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [58]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [59]:
res_buf

In [60]:
res_np

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243394 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898017 , 2.1122162 , 1.2259226 , 2.4700487 , 0.12201902,
        0.72471607, 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.94957584, 1.1395756 , 1.061765  , 1.3816046 , 0.33119798,
        1.1913491 , 0.52872694, 0.81870705, 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [61]:
mult

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [62]:
res_np==mult

array([[ True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False,  True, False,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False,  True,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,

In [63]:
res_np-mult

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         1.1920929e-07,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.1920929e-07, -2.3841858e-07,  0.0000000e+00, -2.3841858e-07,
         0.0000000e+00,  5.9604645e-08,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-5.9604645e-08,  0.0000000e+00,  0.0000000e+00,  1.1920929e-07,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -5.960

In [64]:
res_np.shape

(16, 24)

In [65]:
mult.shape

(16, 24)

# Matmult Dense Dense

In [66]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i; 
        float xval = x[xidx];
        uint yidx = osize*i+gidy;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [67]:
a.shape, b.shape

((16, 32), (32, 24))

In [68]:
rows = a.shape[0]

In [69]:
mult = mult.astype(np.float32)

In [70]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.00 x 0.00 - 0.00  -- 0/0
mult: 0.00 x 0.00 - 0.00  -- 1/24
mult: 0.00 x 0.76 - 0.00  -- 2/48
mult: 0.17 x 0.27 - 0.00  -- 3/72
mult: 0.00 x 0.00 - 0.00  -- 4/96
mult: 0.00 x 0.00 - 0.00  -- 5/120
mult: 0.00 x 0.57 - 0.00  -- 6/144
mult: 0.00 x 0.00 - 0.00  -- 7/168
mult: 0.88 x 0.00 - 0.00  -- 8/192
mult: 0.00 x 0.88 - 0.00  -- 9/216
mult: 0.40 x 0.93 - 0.00  -- 10/240
mult: 0.00 x 0.00 - 0.00  -- 11/264
mult: 0.61 x 0.00 - 0.00  -- 12/288
mult: 0.00 x 0.00 - 0.00  -- 13/312
mult: 0.34 x 0.17 - 0.00  -- 14/336
mult: 0.00 x 0.00 - 0.00  -- 15/360
mult: 0.00 x 0.00 - 0.00  -- 16/384
mult: 0.00 x 0.00 - 0.00  -- 17/408
mult: 0.12 x 0.00 - 0.00  -- 18/432
mult: 0.61 x 0.00 - 0.00  -- 19/456
mult: 0.83 x 0.39 - 0.00  -- 20/480
mult: 0.26 x 0.77 - 0.00  -- 21/504
mult: 0.00 x 0.00 - 0.00  -- 22/528
mult: 0.00 x 0.00 - 0.00  -- 23/552
mult: 0.00 x 0.05 - 0.00  -- 24/576
mult: 0.00 x 0.00 - 0.00  -- 25/600
mult: 0.86 x 0.00 - 0.00  -- 26/624
mult: 0.00 x 0.00 - 0.00  -- 27/648
mult: 0

.79 x 0.00 - 0.00  -- 28/672
mult: 0.00 x 0.42 - 0.00  -- 29/696
mult: 0.14 x 0.15 - 0.00  -- 30/720
mult: 0.00 x 0.00 - 0.00  -- 31/744

In [71]:
(res_np-mult).sum()

1.527369e-07

In [72]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [73]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [74]:
res_np

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243394 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898017 , 2.1122162 , 1.2259226 , 2.4700487 , 0.12201902,
        0.72471607, 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.94957584, 1.1395756 , 1.061765  , 1.3816046 , 0.33119798,
        1.1913491 , 0.52872694, 0.81870705, 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [75]:
a.dot(b)

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [76]:
res_np==mult

array([[ True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False,  True, False,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False,  True,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,

In [77]:
res_np.shape

(16, 24)

In [78]:
mult.shape

(16, 24)

# Matmult Dense Transposed

In [79]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [80]:
c=np.zeros(b.T.shape)
bt = b.T
for row in range(bt.shape[0]):
    for col in range(bt.shape[1]):
        c[row][col] = bt[row][col]

In [81]:
bt

array([[0.        , 0.        , 0.76457417, 0.26847026, 0.        ,
        0.        , 0.5677988 , 0.        , 0.        , 0.8750546 ,
        0.9323805 , 0.        , 0.        , 0.        , 0.16622874,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.39303797, 0.77240014, 0.        , 0.        , 0.04915274,
        0.        , 0.        , 0.        , 0.        , 0.4213172 ,
        0.14627703, 0.        ],
       [0.        , 0.        , 0.5253448 , 0.4472989 , 0.5153698 ,
        0.        , 0.        , 0.        , 0.12423625, 0.        ,
        0.        , 0.        , 0.        , 0.55864817, 0.        ,
        0.        , 0.9435074 , 0.1667168 , 0.12566799, 0.        ,
        0.43565068, 0.4873105 , 0.717685  , 0.57791394, 0.        ,
        0.80733645, 0.        , 0.6129844 , 0.53634363, 0.13901941,
        0.        , 0.        ],
       [0.        , 0.        , 0.58099407, 0.        , 0.71913767,
        0.1559078 , 0.        , 0.48234788, 0.    

In [82]:
c

array([[0.        , 0.        , 0.76457417, 0.26847026, 0.        ,
        0.        , 0.56779879, 0.        , 0.        , 0.8750546 ,
        0.9323805 , 0.        , 0.        , 0.        , 0.16622874,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.39303797, 0.77240014, 0.        , 0.        , 0.04915274,
        0.        , 0.        , 0.        , 0.        , 0.42131719,
        0.14627703, 0.        ],
       [0.        , 0.        , 0.52534479, 0.44729891, 0.51536977,
        0.        , 0.        , 0.        , 0.12423625, 0.        ,
        0.        , 0.        , 0.        , 0.55864817, 0.        ,
        0.        , 0.94350737, 0.1667168 , 0.12566799, 0.        ,
        0.43565068, 0.4873105 , 0.71768498, 0.57791394, 0.        ,
        0.80733645, 0.        , 0.61298442, 0.53634363, 0.13901941,
        0.        , 0.        ],
       [0.        , 0.        , 0.58099407, 0.        , 0.71913767,
        0.15590779, 0.        , 0.48234788, 0.    

In [83]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i;
        float xval = x[xidx];
        uint yidx = msize*gidy+i;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [84]:
a.shape, b.T.shape

((16, 32), (24, 32))

In [85]:
rows = a.shape[0]

In [86]:
mult = mult.astype(np.float32)

In [87]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.00 x 0.00 - 0.00  -- 0/0
mult: 0.00 x 0.00 - 0.00  -- 1/1
mult: 0.00 x 0.76 - 0.00  -- 2/2
mult: 0.17 x 0.27 - 0.00  -- 3/3
mult: 0.00 x 0.00 - 0.00  -- 4/4
mult: 0.00 x 0.00 - 0.00  -- 5/5
mult: 0.00 x 0.57 - 0.00  -- 6/6
mult: 0.00 x 0.00 - 0.00  -- 7/7
mult: 0.88 x 0.00 - 0.00  -- 8/8
mult: 0.00 x 0.88 - 0.00  -- 9/9
mult: 0.40 x 0.93 - 0.00  -- 10/10
mult: 0.00 x 0.00 - 0.00  -- 11/11
mult: 0.61 x 0.00 - 0.00  -- 12/12
mult: 0.00 x 0.00 - 0.00  -- 13/13
mult: 0.34 x 0.17 - 0.00  -- 14/14
mult: 0.00 x 0.00 - 0.00  -- 15/15
mult: 0.00 x 0.00 - 0.00  -- 16/16
mult: 0.00 x 0.00 - 0.00  -- 17/17
mult: 0.12 x 0.00 - 0.00  -- 18/18
mult: 0.61 x 0.00 - 0.00  -- 19/19
mult: 0.83 x 0.39 - 0.00  -- 20/20
mult: 0.26 x 0.77 - 0.00  -- 21/21
mult: 0.00 x 0.00 - 0.00  -- 22/22
mult: 0.00 x 0.00 - 0.00  -- 23/23
mult: 0.00 x 0.05 - 0.00  -- 24/24
mult: 0.00 x 0.00 - 0.00  -- 25/25
mult: 0.86 x 0.00 - 0.00  -- 26/26
mult: 0.00 x 0.00 - 0.00  -- 27/27
mult: 0.79 x 0.00 - 0.00  -- 28/28
mult

In [88]:
(res_np-mult).sum()

1.527369e-07

In [89]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [90]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [91]:
res_np

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243394 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898017 , 2.1122162 , 1.2259226 , 2.4700487 , 0.12201902,
        0.72471607, 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.94957584, 1.1395756 , 1.061765  , 1.3816046 , 0.33119798,
        1.1913491 , 0.52872694, 0.81870705, 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [92]:
a.dot(b)

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [93]:
res_np==mult

array([[ True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False,  True, False,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False,  True,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,

In [94]:
res_np.shape

(16, 24)

In [95]:
mult.shape

(16, 24)

# Matmult Transposed Dense

In [96]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [97]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [98]:
at

array([[0.        , 0.        , 0.16898978, 0.        , 0.7123387 ,
        0.5910008 , 0.        , 0.        , 0.        , 0.06666391,
        0.4507213 , 0.41975817, 0.        , 0.        , 0.        ,
        0.5913447 ],
       [0.        , 0.89125365, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03724516, 0.        ,
        0.42652833, 0.        , 0.74566156, 0.        , 0.        ,
        0.        ],
       [0.        , 0.93762594, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.10916828,
        0.49283606, 0.8300311 , 0.11515573, 0.9487138 , 0.        ,
        0.        ],
       [0.16797899, 0.        , 0.27443814, 0.        , 0.        ,
        0.7437254 , 0.6381501 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.6501674 ,
        0.        ],
       [0.        , 0.03801361, 0.9523248 , 0.46642095, 0.        ,
        0.        , 0.80913323, 

In [99]:
c

array([[0.        , 0.        , 0.16898978, 0.        , 0.71233869,
        0.5910008 , 0.        , 0.        , 0.        , 0.06666391,
        0.45072129, 0.41975817, 0.        , 0.        , 0.        ,
        0.59134471],
       [0.        , 0.89125365, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03724516, 0.        ,
        0.42652833, 0.        , 0.74566156, 0.        , 0.        ,
        0.        ],
       [0.        , 0.93762594, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.10916828,
        0.49283606, 0.8300311 , 0.11515573, 0.94871378, 0.        ,
        0.        ],
       [0.16797899, 0.        , 0.27443814, 0.        , 0.        ,
        0.74372542, 0.6381501 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.65016741,
        0.        ],
       [0.        , 0.03801361, 0.95232481, 0.46642095, 0.        ,
        0.        , 0.80913323, 

In [100]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint isize
                          ) { // LOCAL SHARED BUFFER
      uint osize = get_global_size(0);
      int gidy = get_global_id(0); // row
      
      for (uint gidx = 0; gidx < isize; gidx++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, xidx, yidx);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [101]:
a.shape, b.T.shape

((16, 32), (24, 32))

In [102]:
rows = a.shape[0]

In [103]:
mult = mult.astype(np.float32)

In [104]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [b.shape[1]], None, a_buf, b_buf, res_buf, np.uint32(a.shape[1]), np.uint32(rows))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.00 x 0.00 - 0.00  -- 0/0
mult: 0.00 x 0.00 - 0.00  -- 16/24
mult: 0.00 x 0.76 - 0.00  -- 32/48
mult: 0.17 x 0.27 - 0.05  -- 48/72
mult: 0.00 x 0.00 - 0.05  -- 64/96
mult: 0.00 x 0.00 - 0.05  -- 80/120
mult: 0.00 x 0.57 - 0.05  -- 96/144
mult: 0.00 x 0.00 - 0.05  -- 112/168
mult: 0.88 x 0.00 - 0.05  -- 128/192
mult: 0.00 x 0.88 - 0.05  -- 144/216
mult: 0.40 x 0.93 - 0.42  -- 160/240
mult: 0.00 x 0.00 - 0.42  -- 176/264
mult: 0.61 x 0.00 - 0.42  -- 192/288
mult: 0.00 x 0.00 - 0.42  -- 208/312
mult: 0.34 x 0.17 - 0.47  -- 224/336
mult: 0.00 x 0.00 - 0.47  -- 240/360
mult: 0.00 x 0.00 - 0.47  -- 256/384
mult: 0.00 x 0.00 - 0.47  -- 272/408
mult: 0.12 x 0.00 - 0.47  -- 288/432
mult: 0.61 x 0.00 - 0.47  -- 304/456
mult: 0.83 x 0.39 - 0.80  -- 320/480
mult: 0.26 x 0.77 - 1.00  -- 336/504
mult: 0.00 x 0.00 - 1.00  -- 352/528
mult: 0.00 x 0.00 - 1.00  -- 368/552
mult: 0.00 x 0.05 - 1.00  -- 384/576
mult: 0.00 x 0.00 - 1.00  -- 400/600
mult: 0.86 x 0.00 - 1.00  -- 416/624
mult: 0.00 x 0

In [105]:
(res_np-mult).sum()

1.527369e-07

In [106]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [107]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [108]:
res_np

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243394 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898017 , 2.1122162 , 1.2259226 , 2.4700487 , 0.12201902,
        0.72471607, 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.94957584, 1.1395756 , 1.061765  , 1.3816046 , 0.33119798,
        1.1913491 , 0.52872694, 0.81870705, 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [109]:
a.dot(b)

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [110]:
res_np==mult

array([[ True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False,  True, False,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False,  True,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,

In [111]:
res_np.shape

(16, 24)

In [112]:
mult.shape

(16, 24)

# Matmult Dense Transposed2

In [113]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint osize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      // osize = get_global_size(1);
      int gidx = get_global_id(0); // col
      // int gidy = get_global_id(1); // row

      for (uint gidy = 0; gidy < osize; gidy++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          ret += x[gidx*msize+i]*y[i*osize+gidy];
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f", x[gidx*msize+i],y[i*msize+gidy], ret);
        }

        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [114]:
a.shape, b.shape

((16, 32), (32, 24))

In [115]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [116]:
rows = a.shape[0]

In [117]:
mult = mult.astype(np.float32)

In [118]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]), np.uint32(b.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.00 x 0.00 - 0.00
mult: 0.00 x 0.26 - 0.00
mult: 0.00 x 0.00 - 0.00
mult: 0.17 x 0.00 - 0.05
mult: 0.00 x 0.00 - 0.05
mult: 0.00 x 0.00 - 0.05
mult: 0.00 x 0.00 - 0.05
mult: 0.00 x 0.00 - 0.05
mult: 0.88 x 0.00 - 0.05
mult: 0.00 x 0.00 - 0.05
mult: 0.40 x 0.00 - 0.42
mult: 0.00 x 0.00 - 0.42
mult: 0.61 x 0.00 - 0.42
mult: 0.00 x 0.16 - 0.42
mult: 0.34 x 0.00 - 0.47
mult: 0.00 x 0.39 - 0.47
mult: 0.00 x 0.00 - 0.47
mult: 0.00 x 0.40 - 0.47
mult: 0.12 x 0.05 - 0.47
mult: 0.61 x 0.57 - 0.47
mult: 0.83 x 0.00 - 0.80
mult: 0.26 x 0.00 - 1.00
mult: 0.00 x 0.27 - 1.00
mult: 0.00 x 0.00 - 1.00
mult: 0.00 x 1.02 - 1.00
mult: 0.00 x 0.86 - 1.00
mult: 0.86 x 0.27 - 1.00
mult: 0.00 x 1.21 - 1.00
mult: 0.79 x 1.66 - 1.00
mult: 0.00 x 0.14 - 1.00
mult: 0.14 x 0.56 - 1.02
mult: 0.00 x 1.77 - 1.02

In [119]:
(res_np-mult).sum()

1.527369e-07

In [120]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [121]:
b.T

array([[0.        , 0.        , 0.76457417, 0.26847026, 0.        ,
        0.        , 0.5677988 , 0.        , 0.        , 0.8750546 ,
        0.9323805 , 0.        , 0.        , 0.        , 0.16622874,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.39303797, 0.77240014, 0.        , 0.        , 0.04915274,
        0.        , 0.        , 0.        , 0.        , 0.4213172 ,
        0.14627703, 0.        ],
       [0.        , 0.        , 0.5253448 , 0.4472989 , 0.5153698 ,
        0.        , 0.        , 0.        , 0.12423625, 0.        ,
        0.        , 0.        , 0.        , 0.55864817, 0.        ,
        0.        , 0.9435074 , 0.1667168 , 0.12566799, 0.        ,
        0.43565068, 0.4873105 , 0.717685  , 0.57791394, 0.        ,
        0.80733645, 0.        , 0.6129844 , 0.53634363, 0.13901941,
        0.        , 0.        ],
       [0.        , 0.        , 0.58099407, 0.        , 0.71913767,
        0.1559078 , 0.        , 0.48234788, 0.    

In [122]:
a[0]

array([0.        , 0.        , 0.        , 0.16797899, 0.        ,
       0.        , 0.        , 0.        , 0.87880903, 0.        ,
       0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
       0.        , 0.        , 0.        , 0.12211707, 0.61375475,
       0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
       0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
       0.13828662, 0.        ], dtype=float32)

In [123]:
b.T[0]

array([0.        , 0.        , 0.76457417, 0.26847026, 0.        ,
       0.        , 0.5677988 , 0.        , 0.        , 0.8750546 ,
       0.9323805 , 0.        , 0.        , 0.        , 0.16622874,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.39303797, 0.77240014, 0.        , 0.        , 0.04915274,
       0.        , 0.        , 0.        , 0.        , 0.4213172 ,
       0.14627703, 0.        ], dtype=float32)

In [124]:
res_buf

In [125]:
res_np

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243394 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898017 , 2.1122162 , 1.2259226 , 2.4700487 , 0.12201902,
        0.72471607, 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.94957584, 1.1395756 , 1.061765  , 1.3816046 , 0.33119798,
        1.1913491 , 0.52872694, 0.81870705, 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [126]:
a.dot(b)

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [127]:
res_np==mult

array([[ True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False,  True, False,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False,  True,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,

In [128]:
res_np.shape

(16, 24)

In [129]:
mult.shape

(16, 24)

## Weight update kernel

In [130]:
bs = 4

In [131]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [132]:
dim = 8
topk = 2

x = np.random.rand(bs,dim).astype(np.float32)
y = np.random.rand(bs,dim).astype(np.float32)
x.shape,y.shape, topk

((4, 8), (4, 8), 2)

x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);

      uint idx = n*gid2+gid;

      float valx = x[idx];
      float valy = y[idx];
      uint posx = 0;
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        float tempval2 = y[idx2];
        bool larger = tempval > valx;
        bool larger2 = tempval2 > valy;

        barrier(CLK_GLOBAL_MEM_FENCE);
        posx += (larger)?1:0;
        posy += (larger2)?1:0;
        barrier(CLK_GLOBAL_MEM_FENCE);
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      //printf("posx:%i", posx);
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = gid;
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
          //printf("\\nJ:%i  gid:%i", j, gid);
          //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
          barrier(CLK_GLOBAL_MEM_FENCE);
          
        }
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
    }""").build()

In [133]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#x_cp_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
#x_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#y_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             uint bs,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      //uint bs = get_global_size(1);
      //uint gid2 = get_global_id(1);

      for (uint gid2=0; gid2<bs; gid2++){
        uint idx = n*gid2+gid;

        float valx = x[idx];
        float valy = y[idx];
        uint posx = 0;
        uint posy = 0;
        for (uint i = 0; i < n; i++) {
          uint idx2 = n*gid2+i;
          float tempval = x[idx2];
          float tempval2 = y[idx2];
          bool larger = tempval > valx;
          bool larger2 = tempval2 > valy;

          posx += (larger)?1:0;
          posy += (larger2)?1:0;
        }
        //printf("posx:%i", posx);
        if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
        }
        if (posy < topk) {
          youtidx[posy+topk*gid2] = gid;
        }
      }
      for (uint gid2=0; gid2<bs; gid2++){
        if (gid < topk) {
          for (uint j=0; j<topk; j++) {
            float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
            //printf("\\nJ:%i  gid:%i", j, gid);
            //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
            //barrier(CLK_GLOBAL_MEM_FENCE);
            xout[gid2*topk*topk+j*topk+gid] = res;
          }
        }
      }
    }""").build()

In [134]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
evt = knl(queue, [dim], None, x_buf, y_buf, x_cp_buf, np.uint32(topk), np.uint32(bs), x_idx_buf, y_idx_buf)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

knl(queue, [dim], None, y_buf, x_buf, x_cp_buft, np.uint32(topk), np.uint32(bs), x_idx_buft, y_idx_buft)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [135]:
x

array([[0.6935489 , 0.9508296 , 0.93161595, 0.30010068, 0.6225004 ,
        0.94446224, 0.4452269 , 0.9630355 ],
       [0.12671977, 0.35218495, 0.13177215, 0.82275254, 0.90102214,
        0.80742604, 0.88826716, 0.54051477],
       [0.51984555, 0.08185969, 0.36027512, 0.50712866, 0.8747787 ,
        0.4731751 , 0.37139073, 0.78694844],
       [0.39375284, 0.23605132, 0.8326379 , 0.9267812 , 0.9083676 ,
        0.1486666 , 0.9701302 , 0.05004898]], dtype=float32)

In [136]:
y

array([[0.9854881 , 0.00333519, 0.6735835 , 0.90304184, 0.26640543,
        0.18927108, 0.46144298, 0.44740173],
       [0.49698558, 0.7514179 , 0.7938446 , 0.7909482 , 0.2642381 ,
        0.31813866, 0.23216514, 0.30429915],
       [0.18636614, 0.3030459 , 0.6306314 , 0.8987896 , 0.52226955,
        0.03037781, 0.18171781, 0.39266336],
       [0.907629  , 0.8291435 , 0.48200926, 0.47418508, 0.34457493,
        0.38500836, 0.7870528 , 0.67323244]], dtype=float32)

In [137]:
x.shape, y.shape

((4, 8), (4, 8))

In [138]:
resx

array([0.9490601 , 0.93703127, 0.8696614 , 0.85863894, 0.71527153,
       0.7051461 , 0.71266186, 0.7025733 , 0.78624195, 0.7073011 ,
       0.5516629 , 0.49627438, 0.8805183 , 0.84117347, 0.8043772 ,
       0.7684346 ], dtype=float32)

In [139]:
resx.reshape(bs,topk,topk)

array([[[0.9490601 , 0.93703127],
        [0.8696614 , 0.85863894]],

       [[0.71527153, 0.7051461 ],
        [0.71266186, 0.7025733 ]],

       [[0.78624195, 0.7073011 ],
        [0.5516629 , 0.49627438]],

       [[0.8805183 , 0.84117347],
        [0.8043772 , 0.7684346 ]]], dtype=float32)

In [140]:
resxidx

array([7, 1, 4, 6, 4, 7, 6, 3], dtype=uint32)

In [141]:
resyidx

array([0, 3, 2, 3, 3, 2, 0, 1], dtype=uint32)

In [142]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [143]:
xy0[3][7]

0.2503629

## Weight update kernel new

In [144]:
b

array([[0.        , 0.        , 0.        , 0.        , 0.2106778 ,
        0.6727729 , 0.        , 0.        , 0.        , 0.994735  ,
        0.        , 0.        , 0.        , 0.        , 0.7231555 ,
        0.8717573 , 0.        , 0.00371827, 0.        , 0.47682822,
        0.        , 0.12149344, 0.7363708 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1913773 , 0.        , 0.34261224, 0.26097038, 0.        ,
        0.9146003 , 0.        , 0.96853554, 0.3060288 , 0.        ,
        0.        , 0.10322573, 0.        , 0.03001965, 0.65790516,
        0.        , 0.        , 0.        , 0.        ],
       [0.76457417, 0.5253448 , 0.58099407, 0.48766515, 0.        ,
        0.        , 0.143648  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.80803716, 0.        ,
        0.        , 0.        , 0.8913566 , 0.9567756 , 0.        ,
        0.45077503, 0.        , 0.        , 0.        ],
       [0.268

In [145]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [146]:
at

array([[0.        , 0.        , 0.16898978, 0.        , 0.7123387 ,
        0.5910008 , 0.        , 0.        , 0.        , 0.06666391,
        0.4507213 , 0.41975817, 0.        , 0.        , 0.        ,
        0.5913447 ],
       [0.        , 0.89125365, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03724516, 0.        ,
        0.42652833, 0.        , 0.74566156, 0.        , 0.        ,
        0.        ],
       [0.        , 0.93762594, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.10916828,
        0.49283606, 0.8300311 , 0.11515573, 0.9487138 , 0.        ,
        0.        ],
       [0.16797899, 0.        , 0.27443814, 0.        , 0.        ,
        0.7437254 , 0.6381501 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.6501674 ,
        0.        ],
       [0.        , 0.03801361, 0.9523248 , 0.46642095, 0.        ,
        0.        , 0.80913323, 

In [147]:
c

array([[0.        , 0.        , 0.16898978, 0.        , 0.71233869,
        0.5910008 , 0.        , 0.        , 0.        , 0.06666391,
        0.45072129, 0.41975817, 0.        , 0.        , 0.        ,
        0.59134471],
       [0.        , 0.89125365, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.03724516, 0.        ,
        0.42652833, 0.        , 0.74566156, 0.        , 0.        ,
        0.        ],
       [0.        , 0.93762594, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.10916828,
        0.49283606, 0.8300311 , 0.11515573, 0.94871378, 0.        ,
        0.        ],
       [0.16797899, 0.        , 0.27443814, 0.        , 0.        ,
        0.74372542, 0.6381501 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.65016741,
        0.        ],
       [0.        , 0.03801361, 0.95232481, 0.46642095, 0.        ,
        0.        , 0.80913323, 

a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate3(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             uint topk,
                             uint msize,
                             __global  float* xout,    // INPUT
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);



      uint idx = n*gid2+gid;
      float valx = x[idx];
      uint posx = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        bool larger = tempval > valx;
        posx += (larger)?1:0;
      }
      
      uint idxy = n*gid2+gid;
      float valy = y[idx];
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval2 = y[idx2];
        bool larger2 = tempval2 > valy;
        posy += (larger2)?1:0;
      }
      
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = idx;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = idxy;
      }
      return;
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*msize] * y[youtidx[j+topk*gid2]+gid2*msize];
          printf("\\nJ:%i  gid:(%i,%i)", j, gid, gid2);
          printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          //barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
        }
      }
    }""").build()

In [148]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate3(__global  float* x,     // INPUT MATRIX DATA
                              __global  float* y,    // INPUT
                              __global  float* xsum,    // INPUT
                              __global  float* ysum,    // INPUT
                              uint isize,
                              uint msize,
                              uint osize,
                              uint topk,
                              __global  float* xout,
                              __global  uint* xoutidx,   
                              __global  uint* youtidx    
                              ) { 
      uint gid = get_global_id(0);
      
      // get for a: sum axis0  b: sum axis1 then get topk
      ///////////////////////////////////////////////////
      if (gid < isize) {
        xsum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = x[i*isize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          xsum[gid] += val;
        }
        
        float valx = xsum[gid];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = xsum[i];
          bool larger = tempval > valx;
          posx += (larger)?1:0;
        }
        if (posx < topk) {
          xoutidx[posx] = gid;
        }
      }
      
      if (gid < osize) {
        ysum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = y[i*osize+gid];
          if (gid == 0) {
            printf("\\nADD VALx: %.2f - %i", val, gid*osize+i);
          }
          ysum[gid] += val;
        }
        
        float valy = ysum[gid];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = ysum[i];
          bool larger = tempval > valy;
          posy += (larger)?1:0;
        }
      
        if (posy < topk) {
          youtidx[posy] = gid;
        }
      }
      
      if (gid < topk) {
        float valx = xoutidx[gid];
        uint posx = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = xoutidx[i];
          bool larger = tempval < valx;
          posx += (larger)?1:0;
        }
        xoutidx[posx] = valx;
        
        float valy = youtidx[gid];
        uint posy = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = youtidx[i];
          bool larger = tempval < valy;
          posy += (larger)?1:0;
        }
        youtidx[posy] = valy;
      }
      
      // only calc matrix multiplications for used grads
      ///////////////////////////////////////////////////
      if (gid < topk) {
        uint idxx = xoutidx[gid];
        for (uint j=0; j<topk; j++) {
          uint idxy = youtidx[j];
          xout[j*topk+gid] = 0;
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            xout[j*topk+gid] += x[xidx2] * y[yidx2];
            //if (gid == 0 && j == 1)
            //  printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
        }
      }
    }""").build()

In [149]:
a.shape, b.shape

((16, 32), (32, 24))

In [150]:
rows = a.shape[0]
msize = a.shape[1]

In [151]:
cols = b.shape[1]

In [152]:
mult = a.dot(b)

In [153]:
mult = mult.astype(np.float32)

In [154]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.genwupdate3  # Use this Kernel object for repeated calls
evt = knl(queue, [max(rows,cols)], None, a_buf, b_buf, x_sum_buf, y_sum_buf, np.uint32(rows), np.uint32(msize),np.uint32(cols), np.uint32(topk), x_cp_buf, x_idx_buf, y_idx_buf)

resxsum = np.zeros(a.shape[0]).astype(np.float32)
resysum = np.zeros(b.shape[1]).astype(np.float32)
resx = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, resxsum, x_sum_buf)
cl.enqueue_copy(queue, resysum, y_sum_buf)
cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)


ADD VALx: 0.00 - 0
ADD VALx: 0.00 - 1
ADD VALx: 0.76 - 2
ADD VALx: 0.27 - 3
ADD VALx: 0.00 - 4
ADD VALx: 0.00 - 5
ADD VALx: 0.57 - 6
ADD VALx: 0.00 - 7
ADD VALx: 0.00 - 8
ADD VALx: 0.88 - 9
ADD VALx: 0.93 - 10
ADD VALx: 0.00 - 11
ADD VALx: 0.00 - 12
ADD VALx: 0.00 - 13
ADD VALx: 0.17 - 14
ADD VALx: 0.00 - 15
ADD VALx: 0.00 - 16
ADD VALx: 0.00 - 17
ADD VALx: 0.00 - 18
ADD VALx: 0.00 - 19
ADD VALx: 0.39 - 20
ADD VALx: 0.77 - 21
ADD VALx: 0.00 - 22
ADD VALx: 0.00 - 23
ADD VALx: 0.05 - 24
ADD VALx: 0.00 - 25
ADD VALx: 0.00 - 26
ADD VALx: 0.00 - 27
ADD VALx: 0.00 - 28
ADD VALx: 0.42 - 29
ADD VALx: 0.15 - 30
ADD VALx: 0.00 - 31

In [155]:
resx.reshape(topk,topk)

array([[1.9862504, 1.0541725],
       [1.3941562, 1.567569 ]], dtype=float32)

In [156]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [157]:
b.T

array([[0.        , 0.        , 0.76457417, 0.26847026, 0.        ,
        0.        , 0.5677988 , 0.        , 0.        , 0.8750546 ,
        0.9323805 , 0.        , 0.        , 0.        , 0.16622874,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.39303797, 0.77240014, 0.        , 0.        , 0.04915274,
        0.        , 0.        , 0.        , 0.        , 0.4213172 ,
        0.14627703, 0.        ],
       [0.        , 0.        , 0.5253448 , 0.4472989 , 0.5153698 ,
        0.        , 0.        , 0.        , 0.12423625, 0.        ,
        0.        , 0.        , 0.        , 0.55864817, 0.        ,
        0.        , 0.9435074 , 0.1667168 , 0.12566799, 0.        ,
        0.43565068, 0.4873105 , 0.717685  , 0.57791394, 0.        ,
        0.80733645, 0.        , 0.6129844 , 0.53634363, 0.13901941,
        0.        , 0.        ],
       [0.        , 0.        , 0.58099407, 0.        , 0.71913767,
        0.1559078 , 0.        , 0.48234788, 0.    

In [158]:
resxsum

array([6.0063095, 6.2453794, 5.2608895, 6.9315414, 4.6703515, 6.5347614,
       7.053068 , 5.9966435, 4.5457644, 5.4828196, 5.6722326, 8.010339 ,
       6.658522 , 8.913177 , 5.2867527, 6.0327044], dtype=float32)

In [159]:
resysum

array([5.3566923, 7.7210345, 6.871453 , 5.128611 , 5.338044 , 7.6389756,
       6.2639375, 2.4023628, 7.8125954, 6.8183584, 4.7414207, 4.7794366,
       4.516617 , 6.192719 , 6.5204697, 4.6316047, 2.5904493, 3.9372578,
       4.8595767, 8.443679 , 4.200506 , 5.535129 , 7.7604957, 5.4377947],
      dtype=float32)

In [160]:
a.sum(axis=1)

array([6.0063095, 6.2453785, 5.26089  , 6.931541 , 4.6703515, 6.534762 ,
       7.0530686, 5.9966435, 4.545764 , 5.4828196, 5.672232 , 8.01034  ,
       6.6585217, 8.9131775, 5.286753 , 6.032704 ], dtype=float32)

In [161]:
b.sum(axis=0)

array([5.3566923, 7.7210345, 6.871453 , 5.128611 , 5.338044 , 7.6389756,
       6.2639375, 2.4023628, 7.8125954, 6.8183584, 4.7414207, 4.7794366,
       4.516617 , 6.192719 , 6.5204697, 4.6316047, 2.5904493, 3.9372578,
       4.8595767, 8.443679 , 4.200506 , 5.535129 , 7.7604957, 5.4377947],
      dtype=float32)

In [162]:
mult

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [163]:
resxidx

array([11, 13], dtype=uint32)

In [164]:
resyidx

array([ 8, 19], dtype=uint32)

In [165]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [166]:
xy0[0][0]

0.0629779

## Weight update kernel new2 (sparse ouput)

In [167]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [168]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
sdata_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*topk*4)
sidxs_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*topk*4)
snnzs_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*4)
sdatat_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*topk*4)
sidxst_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*topk*4)
snnzst_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate4(__global  float* x,     // INPUT MATRIX DATA
                              __global  float* y,    // INPUT
                              __global  float* xsum,    // INPUT
                              __global  float* ysum,    // INPUT
                              uint isize,
                              uint msize,
                              uint osize,
                              uint topk,
                              __global  uint*  xoutidx,
                              __global  uint*  youtidx,
                              __global  float* matData,     // INPUT MATRIX DATA
                              __global  uint*  colIdx,
                              __global  uint*  rowNnz,
                              __global  float* matDatat,     // INPUT MATRIX DATA
                              __global  uint*  colIdxt,
                              __global  uint*  rowNnzt
                              ) { 
      uint gid = get_global_id(0);
      
      // get for a: sum axis0  b: sum axis1 then get topk
      ///////////////////////////////////////////////////
      if (gid < isize) {
        xsum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = x[i*isize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          xsum[gid] += val;
        }
        
        float valx = xsum[gid];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = xsum[i];
          bool larger = tempval > valx;
          posx += (larger)?1:0;
        }
        if (posx < topk) {
          xoutidx[posx] = gid;
        }
      }
      
      if (gid < osize) {
        ysum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = y[i*osize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, gid*osize+i);
          //}
          ysum[gid] += val;
        }
        
        float valy = ysum[gid];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = ysum[i];
          bool larger = tempval > valy;
          posy += (larger)?1:0;
        }
      
        if (posy < topk) {
          youtidx[posy] = gid;
        }
      }
      
      if (gid < topk) {
        float valx = xoutidx[gid];
        uint posx = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = xoutidx[i];
          bool larger = tempval < valx;
          posx += (larger)?1:0;
        }
        xoutidx[posx] = valx;
        
        float valy = youtidx[gid];
        uint posy = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = youtidx[i];
          bool larger = tempval < valy;
          posy += (larger)?1:0;
        }
        youtidx[posy] = valy;
      }
      
      // only calc matrix multiplications for used grads
      ///////////////////////////////////////////////////
      if (gid < isize) {
        for (uint i=0; i<topk; i++) {
          matData[gid*topk+i] = 0;
          colIdx[gid*topk+i] = 0;
        }
        rowNnz[gid] = 0;  
      }
      if (gid < osize) {
        for (uint i=0; i<topk; i++) {
          matDatat[gid*topk+i] = 0;
          colIdxt[gid*topk+i] = 0;
        }
        rowNnzt[gid] = 0;  
      }
      
      
      if (gid < topk) {
        uint idxx = xoutidx[gid];
        for (uint j=0; j<topk; j++) {
          uint idxy = youtidx[j];
          //printf("\\nIDXX:%i  IDXY:%i", idxx, idxy);
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            uint colidx = idxy;
            matData[idxx*topk+j] += x[xidx2] * y[yidx2];
            colIdx[idxx*topk+j] = idxy;
            //if (gid == 0)
            //  printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
          rowNnz[idxx] += 1;
        }
      }
      
      if (gid < topk) {
        uint idxy = youtidx[gid];
        for (uint j=0; j<topk; j++) {
          uint idxx = xoutidx[j];
          printf("\\nB-IDXX:%i  IDXY:%i", idxx, idxy);
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            uint colidx = idxy;
            float addval = x[xidx2] * y[yidx2];
            matDatat[idxy*topk+j] += addval;
            colIdxt[idxy*topk+j] = idxx;
            if (gid == 0)
              printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
          rowNnzt[idxy] += 1;
          printf("\\nAdd NNz:%i - %i", idxy, rowNnzt[idxy]);
        }
      }
    }""").build()

In [169]:
a.shape, b.shape

((16, 32), (32, 24))

In [170]:
rows = a.shape[0]
msize = a.shape[1]

In [171]:
cols = b.shape[1]

In [172]:
mult = a.dot(b)

In [173]:
mult = mult.astype(np.float32)

In [174]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.genwupdate4  # Use this Kernel object for repeated calls
evt = knl(queue, [max(rows,cols)], None, a_buf, b_buf, x_sum_buf, y_sum_buf, np.uint32(rows), np.uint32(msize),np.uint32(cols), 
          np.uint32(topk), x_idx_buf, y_idx_buf, sdata_buf, sidxs_buf, snnzs_buf, sdatat_buf, sidxst_buf, snnzst_buf)


B-IDXX:11  IDXY:8
B-IDXX:11  IDXY:19
 ADD VAL:0.42,0.00 - (11,8) - (0,0,0)
 ADD VAL:0.00,0.26 - (11,8) - (0,0,1)
 ADD VAL:0.83,0.00 - (11,8) - (0,0,2)
 ADD VAL:0.00,0.35 - (11,8) - (0,0,3)
 ADD VAL:0.00,0.00 - (11,8) - (0,0,4)
 ADD VAL:0.00,0.00 - (11,8) - (0,0,5)
 ADD VAL:0.94,0.00 - (11,8) - (0,0,6)
 ADD VAL:0.00,0.00 - (11,8) - (0,0,7)
 ADD VAL:0.59,0.00 - (11,8) - (0,0,8)
 ADD VAL:0.00,0.00 - (11,8) - (0,0,9)
 ADD VAL:0.60,0.86 - (11,8) - (0,0,10)
 ADD VAL:0.00,0.00 - (11,8) - (0,0,11)
 ADD VAL:0.28,0.41 - (11,8) - (0,0,12)
 ADD VAL:0.00,0.00 - (11,8) - (0,0,13)
 ADD VAL:0.00,0.00 - (11,8) - (0,0,14)
 ADD VAL:0.50,0.99 - (11,8) - (0,0,15)
 ADD VAL:0.00,0.97 - (11,8) - (0,0,16)
 ADD VAL:0.00,0.16 - (11,8) - (0,0,17)
 ADD VAL:0.00,0.23 - (11,8) - (0,0,18)
 ADD VAL:0.64,0.47 - (11,8) - (0,0,19)
 ADD VAL:0.00,0.09 - (11,8) - (0,0,20)
 ADD VAL:0.00,0.00 - (11,8) - (0,0,21)
 ADD VAL:0.92,0.00 - (11,8) - (0,0,22)
 ADD VAL:0.00,0.80 - (11,8) - (0,0,23)
 ADD VAL:0.58,0.98 - (11,8) - (0,0,2

In [175]:
resxsum = np.zeros(a.shape[0]).astype(np.float32)
resysum = np.zeros(b.shape[1]).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)
resxdat = np.zeros(a.shape[0]*topk).astype(np.float32)
resxcol = np.zeros(a.shape[0]*topk).astype(np.uint32)
resxnnz = np.zeros(a.shape[0]).astype(np.uint32)
resxdatt = np.zeros(b.shape[1]*topk).astype(np.float32)
resxcolt = np.zeros(b.shape[1]*topk).astype(np.uint32)
resxnnzt = np.zeros(b.shape[1]).astype(np.uint32)

cl.enqueue_copy(queue, resxsum, x_sum_buf)
cl.enqueue_copy(queue, resysum, y_sum_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)
cl.enqueue_copy(queue, resxdat, sdata_buf)
cl.enqueue_copy(queue, resxcol, sidxs_buf)
cl.enqueue_copy(queue, resxnnz, snnzs_buf)
cl.enqueue_copy(queue, resxdatt, sdatat_buf)
cl.enqueue_copy(queue, resxcolt, sidxst_buf)
cl.enqueue_copy(queue, resxnnzt, snnzst_buf)

## results

In [176]:
mult.T

array([[1.0214844 , 1.5898018 , 0.9495759 , 0.5750399 , 1.2057824 ,
        1.1399653 , 0.5742301 , 0.52133816, 0.561906  , 0.5497874 ,
        1.3367468 , 1.8711118 , 1.0844047 , 1.3107877 , 1.7805773 ,
        0.9499581 ],
       [1.1133547 , 2.1122165 , 1.1395756 , 1.5720973 , 0.5319828 ,
        0.92996156, 2.3780043 , 1.1806033 , 0.65674967, 2.1608338 ,
        0.92636836, 1.67947   , 1.5173159 , 2.5673425 , 1.8103852 ,
        1.9642442 ],
       [0.5297862 , 1.2259226 , 1.061765  , 1.0267305 , 0.6204409 ,
        1.742974  , 1.6552789 , 1.3076915 , 0.881607  , 0.5663832 ,
        0.62990445, 1.2643336 , 0.74794316, 3.525021  , 0.29620373,
        0.8810221 ],
       [1.0237993 , 2.470049  , 1.3816044 , 1.7645876 , 0.92333406,
        0.25080562, 0.356461  , 0.7986879 , 1.2948544 , 1.8157119 ,
        1.5882485 , 1.2340922 , 0.5202013 , 2.225675  , 0.24450383,
        0.2648582 ],
       [1.7243392 , 0.12201902, 0.33119798, 0.56259596, 1.0426908 ,
        1.5223929 , 0.3567271 , 

In [177]:
resxdat.reshape(a.shape[0],topk)

array([[0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [1.9862504, 1.3941562],
       [0.       , 0.       ],
       [1.0541725, 1.567569 ],
       [0.       , 0.       ],
       [0.       , 0.       ]], dtype=float32)

In [178]:
resxcol.reshape(a.shape[0],topk)

array([[ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 8, 19],
       [ 0,  0],
       [ 8, 19],
       [ 0,  0],
       [ 0,  0]], dtype=uint32)

In [179]:
resxnnz.reshape(a.shape[0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0], dtype=uint32)

In [180]:
resxdatt.reshape(b.shape[1],topk)

array([[0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [1.9862504, 1.0541725],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [1.3941562, 1.567569 ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ]], dtype=float32)

In [181]:
resxcolt.reshape(b.shape[1],topk)

array([[ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [11, 13],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [11, 13],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0]], dtype=uint32)

In [182]:
resxnnzt.reshape(b.shape[1])

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0], dtype=uint32)

## comp

In [183]:
resxsum

array([6.0063095, 6.2453794, 5.2608895, 6.9315414, 4.6703515, 6.5347614,
       7.053068 , 5.9966435, 4.5457644, 5.4828196, 5.6722326, 8.010339 ,
       6.658522 , 8.913177 , 5.2867527, 6.0327044], dtype=float32)

In [184]:
a.sum(axis=1)

array([6.0063095, 6.2453785, 5.26089  , 6.931541 , 4.6703515, 6.534762 ,
       7.0530686, 5.9966435, 4.545764 , 5.4828196, 5.672232 , 8.01034  ,
       6.6585217, 8.9131775, 5.286753 , 6.032704 ], dtype=float32)

In [185]:
resysum

array([5.3566923, 7.7210345, 6.871453 , 5.128611 , 5.338044 , 7.6389756,
       6.2639375, 2.4023628, 7.8125954, 6.8183584, 4.7414207, 4.7794366,
       4.516617 , 6.192719 , 6.5204697, 4.6316047, 2.5904493, 3.9372578,
       4.8595767, 8.443679 , 4.200506 , 5.535129 , 7.7604957, 5.4377947],
      dtype=float32)

In [186]:
b.sum(axis=0)

array([5.3566923, 7.7210345, 6.871453 , 5.128611 , 5.338044 , 7.6389756,
       6.2639375, 2.4023628, 7.8125954, 6.8183584, 4.7414207, 4.7794366,
       4.516617 , 6.192719 , 6.5204697, 4.6316047, 2.5904493, 3.9372578,
       4.8595767, 8.443679 , 4.200506 , 5.535129 , 7.7604957, 5.4377947],
      dtype=float32)

In [187]:
mult

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [188]:
resxidx

array([11, 13], dtype=uint32)

In [189]:
resyidx

array([ 8, 19], dtype=uint32)

In [190]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [191]:
xy0[0][0]

0.0629779

### Update Vals (add sparse)

In [192]:
multdata, multcols, multnnz, multellw = to_data(mult)
multdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multdata)
multcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multcols)
multnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multnnz)


prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddense(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            __global  float* matDataAdd,     // INPUT MATRIX DATA
                            __global  uint*  colIdxAdd,
                            __global  uint*  rowNnzAdd,
                            uint ellwidthAdd
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];
      
      uint baseidxs = gid*ellwidth;
      uint baseidxd = gid*ellwidthAdd;
      
      uint nnzadd = rowNnzAdd[gid];
      printf("\\nNNZs: %i   GID:%i", nnzadd, gid);
      
      for (uint i=0; i<nnzadd; i++) {
        float addval = matDataAdd[baseidxd+i];
        uint addcol = colIdxAdd[baseidxd+i];
        
        uint refcol = colIdx[baseidxs+i];
        uint m = 0;
        while (addcol > refcol) {
          m += 1;
          refcol = colIdx[baseidxs+i+m];
        }
        
        //printf("\\nADD VAL:%.2f  ADDCOL:%i  idxs/d:(%i/%i)  gid/i:(%i/%i)", addval, addcol, baseidxs, baseidxd, gid,i);
        if (addval == 0.0) {
          //printf("\\nZERO VAL, CONT: %.2f - %i", addval, gid);
          continue;
        }
        if (addcol == refcol) {
          matData[baseidxs+i+m] += addval;
          printf("\\nINCREMENT: %.2f",addval);
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (addcol > refcol) {
            rowNnz[gid] += 1;
            printf("\\nSET VAL0:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i+m] = addval;
            colIdx[baseidxs+i+m] = addcol;
            continue;
          }
          for (uint j=nnz; j>i+m; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          
          printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i+m] = addval;
          colIdx[baseidxs+i+m] = addcol;
          if (nnz >= ellwidth)
            break;
        }
      }
    }""").build()

In [193]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [194]:
rows = a.shape[0]

In [195]:
mult = mult.astype(np.float32)

In [196]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, multdata_buf, multcols_buf, multnnzs_buf, np.float32(1), np.uint32(multellw), 
    sdata_buf, sidxs_buf, snnzs_buf, np.uint32(topk))


NNZs: 0   GID:0
NNZs: 0   GID:1
NNZs: 0   GID:2
NNZs: 0   GID:3
NNZs: 0   GID:4
NNZs: 0   GID:5
NNZs: 0   GID:6
NNZs: 0   GID:7
NNZs: 0   GID:8
NNZs: 0   GID:9
NNZs: 0   GID:10
NNZs: 2   GID:11
NNZs: 0   GID:12
NNZs: 2   GID:13
NNZs: 0   GID:14
NNZs: 0   GID:15
INCREMENT: 1.99
INCREMENT: 1.05
INCREMENT: 1.39
INCREMENT: 1.57

In [197]:
mult

array([[1.0214844 , 1.1133547 , 0.5297862 , 1.0237993 , 1.7243392 ,
        2.8457184 , 1.1962043 , 0.6928725 , 1.034317  , 1.2665485 ,
        0.7275173 , 0.25065187, 0.7494863 , 0.9242941 , 0.7610628 ,
        2.2162373 , 0.27569565, 1.0976613 , 0.66344446, 1.8264639 ,
        1.4665459 , 0.7415181 , 1.5544621 , 0.13843258],
       [1.5898018 , 2.1122165 , 1.2259226 , 2.470049  , 0.12201902,
        0.724716  , 0.65624374, 0.79921037, 0.86317337, 0.30282417,
        0.9219097 , 0.30281866, 1.8316078 , 1.9282674 , 1.3870243 ,
        0.6804428 , 0.44406316, 0.83575904, 1.9771178 , 1.8887583 ,
        0.7226437 , 1.0255347 , 1.3046051 , 1.1810291 ],
       [0.9495759 , 1.1395756 , 1.061765  , 1.3816044 , 0.33119798,
        1.1913491 , 0.52872694, 0.8187071 , 0.68683094, 1.0772579 ,
        0.71349496, 0.74350876, 0.94780594, 0.1504721 , 1.4588463 ,
        1.0758466 , 0.27213624, 0.0357363 , 1.1459256 , 1.5839037 ,
        0.9332375 , 1.3382342 , 1.1085682 , 1.4414737 ],
       [0.575

In [198]:
data_res = np.empty_like(multdata)
cols_res = np.empty_like(multcols)
nnzs_res = np.empty_like(multnnz)
cl.enqueue_copy(queue, data_res, multdata_buf, is_blocking=True)
cl.enqueue_copy(queue, cols_res, multcols_buf, is_blocking=True)
cl.enqueue_copy(queue, nnzs_res, multnnzs_buf, is_blocking=True)

In [199]:
adenseadd = to_dense(data_res, cols_res, nnzs_res, multellw, mult.shape)
adenseadd.T

array([[1.02148438, 1.58980179, 0.9495759 , 0.57503992, 1.20578241,
        1.1399653 , 0.57423007, 0.52133816, 0.56190598, 0.5497874 ,
        1.33674681, 1.87111175, 1.08440471, 1.31078768, 1.7805773 ,
        0.94995809],
       [1.11335468, 2.11221647, 1.1395756 , 1.5720973 , 0.53198278,
        0.92996156, 2.37800431, 1.18060327, 0.65674967, 2.16083384,
        0.92636836, 1.67946994, 1.51731586, 2.56734252, 1.81038523,
        1.96424425],
       [0.52978623, 1.22592258, 1.06176496, 1.02673054, 0.6204409 ,
        1.74297404, 1.65527892, 1.30769145, 0.881607  , 0.56638318,
        0.62990445, 1.26433361, 0.74794316, 3.52502108, 0.29620373,
        0.8810221 ],
       [1.0237993 , 2.4700489 , 1.38160443, 1.76458764, 0.92333406,
        0.25080562, 0.35646099, 0.79868788, 1.2948544 , 1.81571186,
        1.58824849, 1.23409224, 0.52020133, 2.22567511, 0.24450383,
        0.26485819],
       [1.72433925, 0.12201902, 0.33119798, 0.56259596, 1.04269075,
        1.52239287, 0.35672709, 

In [200]:
mult-adenseadd

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
  

### Update Vals (add sparset)

In [201]:
multdata, multcols, multnnz, multellw = to_data(mult)
multdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multdata)
multcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multcols)
multnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multnnz)


prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddense(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            __global  float* matDataAdd,     // INPUT MATRIX DATA
                            __global  uint*  colIdxAdd,
                            __global  uint*  rowNnzAdd,
                            uint ellwidthAdd
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];
      
      uint baseidxs = gid*ellwidth;
      uint baseidxd = gid*ellwidthAdd;
      
      uint nnzadd = rowNnzAdd[gid];
      printf("\\nNNZs: %i   GID:%i", nnzadd, gid);
      
      
      for (uint i=0; i<nnzadd; i++) {
        float addval = 0;//matDataAdd[baseidxd+i];
        uint addcol = colIdxAdd[baseidxd+i];
        
        uint refcol = colIdx[baseidxs+i];
        uint m = 0;
        while (addcol > refcol) {
          m += 1;
          refcol = colIdx[baseidxs+i+m];
        }
        
        //printf("\\nADD VAL:%.2f  ADDCOL:%i  idxs/d:(%i/%i)  gid/i:(%i/%i)", addval, addcol, baseidxs, baseidxd, gid,i);
        if (addval == 0.0) {
          //printf("\\nZERO VAL, CONT: %.2f - %i", addval, gid);
          continue;
        }
        if (addcol == refcol) {
          matData[baseidxs+i+m] += addval;
          printf("\\nINCREMENT: %.2f",addval);
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (addcol > refcol) {
            rowNnz[gid] += 1;
            printf("\\nSET VAL0:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i+m] = addval;
            colIdx[baseidxs+i+m] = addcol;
            continue;
          }
          for (uint j=nnz; j>i+m; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          
          printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i+m] = addval;
          colIdx[baseidxs+i+m] = addcol;
          if (nnz >= ellwidth)
            break;
        }
      }
    }""").build()

In [202]:
multt=np.zeros(mult.T.shape)

for row in range(multt.shape[0]):
    for col in range(multt.shape[1]):
        multt[row][col] = mult[col][row]

In [203]:
multdata, multcols, multnnz, multellw = to_data(multt)
multdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multdata)
multcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multcols)
multnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=multnnz)

In [204]:
a.shape, b.shape

((16, 32), (32, 24))

In [205]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [206]:
rows = mult.T.shape[0]

In [207]:
mult = mult.astype(np.float32)

In [208]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, multdata_buf, multcols_buf, multnnzs_buf, np.float32(1), np.uint32(multellw), 
    sdatat_buf, sidxst_buf, snnzst_buf, np.uint32(topk))


NNZs: 0   GID:0
NNZs: 0   GID:1
NNZs: 0   GID:2
NNZs: 0   GID:3
NNZs: 0   GID:4
NNZs: 0   GID:5
NNZs: 0   GID:6
NNZs: 0   GID:7
NNZs: 2   GID:8
NNZs: 0   GID:9
NNZs: 0   GID:10
NNZs: 0   GID:11
NNZs: 0   GID:12
NNZs: 0   GID:13
NNZs: 0   GID:14
NNZs: 0   GID:15
NNZs: 0   GID:16
NNZs: 0   GID:17
NNZs: 0   GID:18
NNZs: 2   GID:19
NNZs: 0   GID:20
NNZs: 0   GID:21
NNZs: 0   GID:22
NNZs: 0   GID:23

In [209]:
mult.T

array([[1.0214844 , 1.5898018 , 0.9495759 , 0.5750399 , 1.2057824 ,
        1.1399653 , 0.5742301 , 0.52133816, 0.561906  , 0.5497874 ,
        1.3367468 , 1.8711118 , 1.0844047 , 1.3107877 , 1.7805773 ,
        0.9499581 ],
       [1.1133547 , 2.1122165 , 1.1395756 , 1.5720973 , 0.5319828 ,
        0.92996156, 2.3780043 , 1.1806033 , 0.65674967, 2.1608338 ,
        0.92636836, 1.67947   , 1.5173159 , 2.5673425 , 1.8103852 ,
        1.9642442 ],
       [0.5297862 , 1.2259226 , 1.061765  , 1.0267305 , 0.6204409 ,
        1.742974  , 1.6552789 , 1.3076915 , 0.881607  , 0.5663832 ,
        0.62990445, 1.2643336 , 0.74794316, 3.525021  , 0.29620373,
        0.8810221 ],
       [1.0237993 , 2.470049  , 1.3816044 , 1.7645876 , 0.92333406,
        0.25080562, 0.356461  , 0.7986879 , 1.2948544 , 1.8157119 ,
        1.5882485 , 1.2340922 , 0.5202013 , 2.225675  , 0.24450383,
        0.2648582 ],
       [1.7243392 , 0.12201902, 0.33119798, 0.56259596, 1.0426908 ,
        1.5223929 , 0.3567271 , 

In [210]:
data_res = np.empty_like(multdata)
cols_res = np.empty_like(multcols)
nnzs_res = np.empty_like(multnnz)
cl.enqueue_copy(queue, data_res, multdata_buf, is_blocking=True)
cl.enqueue_copy(queue, cols_res, multcols_buf, is_blocking=True)
cl.enqueue_copy(queue, nnzs_res, multnnzs_buf, is_blocking=True)

In [211]:
multt-data_res.reshape(multt.shape)

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0

In [212]:
nnzs_res

array([16, 16, 16, 16, 16, 16, 16, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16], dtype=uint32)

In [213]:
adenseaddt = to_dense(data_res, cols_res, nnzs_res, multellw, multt.shape)
adenseaddt

array([[1.02148438, 1.58980179, 0.9495759 , 0.57503992, 1.20578241,
        1.1399653 , 0.57423007, 0.52133816, 0.56190598, 0.5497874 ,
        1.33674681, 1.87111175, 1.08440471, 1.31078768, 1.7805773 ,
        0.94995809],
       [1.11335468, 2.11221647, 1.1395756 , 1.5720973 , 0.53198278,
        0.92996156, 2.37800431, 1.18060327, 0.65674967, 2.16083384,
        0.92636836, 1.67946994, 1.51731586, 2.56734252, 1.81038523,
        1.96424425],
       [0.52978623, 1.22592258, 1.06176496, 1.02673054, 0.6204409 ,
        1.74297404, 1.65527892, 1.30769145, 0.881607  , 0.56638318,
        0.62990445, 1.26433361, 0.74794316, 3.52502108, 0.29620373,
        0.8810221 ],
       [1.0237993 , 2.4700489 , 1.38160443, 1.76458764, 0.92333406,
        0.25080562, 0.35646099, 0.79868788, 1.2948544 , 1.81571186,
        1.58824849, 1.23409224, 0.52020133, 2.22567511, 0.24450383,
        0.26485819],
       [1.72433925, 0.12201902, 0.33119798, 0.56259596, 1.04269075,
        1.52239287, 0.35672709, 

In [214]:
multt-adenseaddt

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [215]:
adenseaddt

array([[1.02148438, 1.58980179, 0.9495759 , 0.57503992, 1.20578241,
        1.1399653 , 0.57423007, 0.52133816, 0.56190598, 0.5497874 ,
        1.33674681, 1.87111175, 1.08440471, 1.31078768, 1.7805773 ,
        0.94995809],
       [1.11335468, 2.11221647, 1.1395756 , 1.5720973 , 0.53198278,
        0.92996156, 2.37800431, 1.18060327, 0.65674967, 2.16083384,
        0.92636836, 1.67946994, 1.51731586, 2.56734252, 1.81038523,
        1.96424425],
       [0.52978623, 1.22592258, 1.06176496, 1.02673054, 0.6204409 ,
        1.74297404, 1.65527892, 1.30769145, 0.881607  , 0.56638318,
        0.62990445, 1.26433361, 0.74794316, 3.52502108, 0.29620373,
        0.8810221 ],
       [1.0237993 , 2.4700489 , 1.38160443, 1.76458764, 0.92333406,
        0.25080562, 0.35646099, 0.79868788, 1.2948544 , 1.81571186,
        1.58824849, 1.23409224, 0.52020133, 2.22567511, 0.24450383,
        0.26485819],
       [1.72433925, 0.12201902, 0.33119798, 0.56259596, 1.04269075,
        1.52239287, 0.35672709, 

In [216]:
adenseadd.T == adenseaddt

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

### Update Vals (add dense)

In [217]:
matadd = np.random.randn(*a.shape).astype(np.float32)
matadd

array([[-6.26268864e-01,  1.16298282e+00,  2.48817801e-01,
        -1.44098580e+00, -4.47880700e-02,  1.89448595e+00,
         1.05995822e+00, -1.34446847e+00,  6.68461144e-01,
         2.06624222e+00,  1.02297318e+00,  4.73099858e-01,
         7.68838108e-01,  2.23700786e+00, -7.76244223e-01,
        -9.05630112e-01, -1.68662882e+00, -3.59342635e-01,
         7.55411983e-01, -4.72951472e-01,  3.92256171e-01,
         7.15914905e-01, -8.42166245e-02,  1.31267536e+00,
        -9.60130692e-02,  6.88466311e-01, -1.60180056e+00,
         4.34886247e-01,  2.15700984e-01, -1.03147888e+00,
        -1.71604469e-01,  8.94104123e-01],
       [ 4.76273358e-01, -2.50430346e+00, -7.88759351e-01,
        -1.83206809e+00, -1.36077106e-01, -7.99689770e-01,
         3.63241166e-01,  7.21472740e-01,  1.47591758e+00,
         4.40313458e-01,  1.88291383e+00,  8.21732637e-03,
         8.68482232e-01,  5.57532609e-01, -1.76629674e+00,
         2.14440060e+00, -6.36306465e-01,  1.80682361e+00,
        -2.78

In [218]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [219]:
a_added = a + matadd

In [220]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddense(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   awidth,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      uint baseidxd = gid*awidth;
      
      for (uint i=0; i<awidth; i++) {
        float addval = vector_x[baseidxd+i];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          matData[baseidxs+i] += addval;
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            continue;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
          if (nnz >= ellwidth)
            break;
        }
      }
    }""").build()

In [221]:
a.shape, b.shape

((16, 32), (32, 24))

In [222]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [223]:
rows = a.shape[0]

In [224]:
mult = mult.astype(np.float32)

In [225]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa),np.uint32(a.shape[1]), add_buf)

In [226]:
matadd[0][0]

-0.62626886

In [227]:
data_res = np.empty_like(adata)
cols_res = np.empty_like(acols)
nnzs_res = np.empty_like(annz)
cl.enqueue_copy(queue, data_res, adata_buf)
cl.enqueue_copy(queue, cols_res, acols_buf)
cl.enqueue_copy(queue, nnzs_res, annzs_buf)

In [228]:
adenseadd = to_dense(data_res, cols_res, nnzs_res, ellwa, a.shape)
adenseadd

array([[-0.62626886,  1.16298282,  0.2488178 , -1.2730068 , -0.04478807,
         0.        ,  0.        ,  0.        ,  0.87880903,  0.        ,
         0.4000462 ,  0.        ,  0.60551333,  0.        ,  0.33975551,
         0.        ,  0.        ,  0.        ,  0.12211707,  0.61375475,
         0.82977319,  0.25965056,  0.        ,  0.        ,  0.        ,
         0.        ,  0.85697263,  0.        ,  0.79365158,  0.        ,
         0.13828662,  0.        ],
       [ 0.47627336, -1.61304975,  0.14886659, -1.83206809, -0.09806349,
        -0.79968977,  0.36324117,  0.        ,  0.13324429,  0.        ,
         0.        ,  0.35636038,  0.        ,  0.40821061,  0.        ,
         0.        ,  0.        ,  0.98059231,  0.        ,  0.        ,
         0.        ,  0.67875183,  0.478064  ,  0.        ,  0.        ,
         0.44724301,  0.        ,  0.06849757,  0.        ,  0.82752186,
         0.        ,  0.        ],
       [-2.94134665,  1.69544852,  0.50442356,  0.7352

In [229]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [230]:
matadd

array([[-6.26268864e-01,  1.16298282e+00,  2.48817801e-01,
        -1.44098580e+00, -4.47880700e-02,  1.89448595e+00,
         1.05995822e+00, -1.34446847e+00,  6.68461144e-01,
         2.06624222e+00,  1.02297318e+00,  4.73099858e-01,
         7.68838108e-01,  2.23700786e+00, -7.76244223e-01,
        -9.05630112e-01, -1.68662882e+00, -3.59342635e-01,
         7.55411983e-01, -4.72951472e-01,  3.92256171e-01,
         7.15914905e-01, -8.42166245e-02,  1.31267536e+00,
        -9.60130692e-02,  6.88466311e-01, -1.60180056e+00,
         4.34886247e-01,  2.15700984e-01, -1.03147888e+00,
        -1.71604469e-01,  8.94104123e-01],
       [ 4.76273358e-01, -2.50430346e+00, -7.88759351e-01,
        -1.83206809e+00, -1.36077106e-01, -7.99689770e-01,
         3.63241166e-01,  7.21472740e-01,  1.47591758e+00,
         4.40313458e-01,  1.88291383e+00,  8.21732637e-03,
         8.68482232e-01,  5.57532609e-01, -1.76629674e+00,
         2.14440060e+00, -6.36306465e-01,  1.80682361e+00,
        -2.78

In [231]:
a_added

array([[-6.26268864e-01,  1.16298282e+00,  2.48817801e-01,
        -1.27300680e+00, -4.47880700e-02,  1.89448595e+00,
         1.05995822e+00, -1.34446847e+00,  1.54727018e+00,
         2.06624222e+00,  1.42301941e+00,  4.73099858e-01,
         1.37435150e+00,  2.23700786e+00, -4.36488718e-01,
        -9.05630112e-01, -1.68662882e+00, -3.59342635e-01,
         8.77529025e-01,  1.40803277e-01,  1.22202933e+00,
         9.75565434e-01, -8.42166245e-02,  1.31267536e+00,
        -9.60130692e-02,  6.88466311e-01, -7.44827926e-01,
         4.34886247e-01,  1.00935256e+00, -1.03147888e+00,
        -3.33178490e-02,  8.94104123e-01],
       [ 4.76273358e-01, -1.61304975e+00,  1.48866594e-01,
        -1.83206809e+00, -9.80634913e-02, -7.99689770e-01,
         3.63241166e-01,  7.21472740e-01,  1.60916185e+00,
         4.40313458e-01,  1.88291383e+00,  3.64577711e-01,
         8.68482232e-01,  9.65743184e-01, -1.76629674e+00,
         2.14440060e+00, -6.36306465e-01,  2.78741598e+00,
        -2.78

In [232]:
adenseadd == a_added

array([[ True,  True,  True,  True,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False],
       [ True,  True,  True,  True,  True,  True,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False],
       [ True,  True,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, F

### update vals

In [233]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddenset(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   aheight,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint ncols = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      
      for (uint i=0; i<aheight; i++) {
        if (nnz > ellwidth)
            break;
        uint baseidxd = i*ncols+gid;
        float addval = vector_x[baseidxd];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] += addval;
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            continue;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
        }
      }
    }""").build()

In [234]:
a.shape, b.shape

((16, 32), (32, 24))

In [235]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [236]:
cols = a.shape[1]

In [237]:
mult = mult.astype(np.float32)

In [238]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddenset  # Use this Kernel object for repeated calls
knl(queue, [cols], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat),np.uint32(a.T.shape[1]), add_buf)


ADD VAL:-1.44 idx:48/3  col:2
ADD VAL:0.67 idx:128/8  col:2
ADD VAL:1.02 idx:160/10  col:2
ADD VAL:0.77 idx:192/12  col:2
ADD VAL:-0.78 idx:224/14  col:2
ADD VAL:0.76 idx:288/18  col:2
ADD VAL:-0.47 idx:304/19  col:2
ADD VAL:0.39 idx:320/20  col:2
ADD VAL:0.72 idx:336/21  col:2
ADD VAL:-1.60 idx:416/26  col:2
ADD VAL:0.22 idx:448/28  col:2
ADD VAL:-0.17 idx:480/30  col:2
ADD VAL:-2.50 idx:17/34  col:2
ADD VAL:-0.79 idx:33/35  col:2
ADD VAL:-0.14 idx:65/37  col:2
ADD VAL:1.48 idx:129/41  col:2
ADD VAL:0.01 idx:177/44  col:2
ADD VAL:0.56 idx:209/46  col:2
ADD VAL:1.81 idx:273/50  col:2
ADD VAL:-0.58 idx:337/54  col:2
ADD VAL:-1.08 idx:353/55  col:2
ADD VAL:-1.05 idx:401/58  col:2
ADD VAL:-0.99 idx:433/60  col:2
ADD VAL:-2.16 idx:465/62  col:2
ADD VAL:-3.11 idx:2/66  col:2
ADD VAL:0.46 idx:50/69  col:2
ADD VAL:1.17 idx:66/70  col:2
ADD VAL:-0.39 idx:82/71  col:2
ADD VAL:0.29 idx:130/74  col:2
ADD VAL:0.91 idx:162/76  col:2
ADD VAL:0.38 idx:178/77  col:2
ADD VAL:-0.98 idx:226/80  col:2
AD

In [239]:
matadd[0][0]

-0.62626886

In [240]:
datat_res = np.empty_like(adatat)
colst_res = np.empty_like(acolst)
nnzst_res = np.empty_like(annzt)
cl.enqueue_copy(queue, datat_res, adatat_buf)
cl.enqueue_copy(queue, colst_res, acolst_buf)
cl.enqueue_copy(queue, nnzst_res, annzst_buf)

In [241]:
adenseaddt = to_dense(datat_res, colst_res, nnzst_res, ellwat, a.T.shape).T
adenseaddt

array([[-6.26268864e-01,  1.16298282e+00,  2.48817801e-01,
        -1.27300680e+00, -4.47880700e-02,  1.89448595e+00,
         1.05995822e+00, -1.34446847e+00,  1.54727018e+00,
         2.06624222e+00,  1.42301941e+00,  4.73099858e-01,
         1.37435150e+00,  2.23700786e+00, -4.36488718e-01,
        -9.05630112e-01, -1.68662882e+00, -3.59342635e-01,
         8.77529025e-01,  1.40803277e-01,  1.22202933e+00,
         9.75565434e-01, -8.42166245e-02,  1.31267536e+00,
        -9.60130692e-02,  6.88466311e-01, -7.44827926e-01,
         4.34886247e-01,  1.00935256e+00, -1.03147888e+00,
        -3.33178490e-02,  8.94104123e-01],
       [ 4.76273358e-01, -1.61304975e+00,  1.48866594e-01,
        -1.83206809e+00, -9.80634913e-02, -7.99689770e-01,
         3.63241166e-01,  7.21472740e-01,  1.60916185e+00,
         4.40313458e-01,  1.88291383e+00,  3.64577711e-01,
         8.68482232e-01,  9.65743184e-01, -1.76629674e+00,
         2.14440060e+00, -6.36306465e-01,  2.78741598e+00,
        -2.78

In [242]:
a

array([[0.        , 0.        , 0.        , 0.16797899, 0.        ,
        0.        , 0.        , 0.        , 0.87880903, 0.        ,
        0.4000462 , 0.        , 0.60551333, 0.        , 0.3397555 ,
        0.        , 0.        , 0.        , 0.12211707, 0.61375475,
        0.8297732 , 0.25965056, 0.        , 0.        , 0.        ,
        0.        , 0.85697263, 0.        , 0.7936516 , 0.        ,
        0.13828662, 0.        ],
       [0.        , 0.89125365, 0.93762594, 0.        , 0.03801361,
        0.        , 0.        , 0.        , 0.13324429, 0.        ,
        0.        , 0.35636038, 0.        , 0.4082106 , 0.        ,
        0.        , 0.        , 0.9805923 , 0.        , 0.        ,
        0.        , 0.6787518 , 0.478064  , 0.        , 0.        ,
        0.447243  , 0.        , 0.06849757, 0.        , 0.82752186,
        0.        , 0.        ],
       [0.16898978, 0.        , 0.        , 0.27443814, 0.9523248 ,
        0.35010964, 0.        , 0.        , 0.1215

In [243]:
matadd

array([[-6.26268864e-01,  1.16298282e+00,  2.48817801e-01,
        -1.44098580e+00, -4.47880700e-02,  1.89448595e+00,
         1.05995822e+00, -1.34446847e+00,  6.68461144e-01,
         2.06624222e+00,  1.02297318e+00,  4.73099858e-01,
         7.68838108e-01,  2.23700786e+00, -7.76244223e-01,
        -9.05630112e-01, -1.68662882e+00, -3.59342635e-01,
         7.55411983e-01, -4.72951472e-01,  3.92256171e-01,
         7.15914905e-01, -8.42166245e-02,  1.31267536e+00,
        -9.60130692e-02,  6.88466311e-01, -1.60180056e+00,
         4.34886247e-01,  2.15700984e-01, -1.03147888e+00,
        -1.71604469e-01,  8.94104123e-01],
       [ 4.76273358e-01, -2.50430346e+00, -7.88759351e-01,
        -1.83206809e+00, -1.36077106e-01, -7.99689770e-01,
         3.63241166e-01,  7.21472740e-01,  1.47591758e+00,
         4.40313458e-01,  1.88291383e+00,  8.21732637e-03,
         8.68482232e-01,  5.57532609e-01, -1.76629674e+00,
         2.14440060e+00, -6.36306465e-01,  1.80682361e+00,
        -2.78

In [244]:
a_added

array([[-6.26268864e-01,  1.16298282e+00,  2.48817801e-01,
        -1.27300680e+00, -4.47880700e-02,  1.89448595e+00,
         1.05995822e+00, -1.34446847e+00,  1.54727018e+00,
         2.06624222e+00,  1.42301941e+00,  4.73099858e-01,
         1.37435150e+00,  2.23700786e+00, -4.36488718e-01,
        -9.05630112e-01, -1.68662882e+00, -3.59342635e-01,
         8.77529025e-01,  1.40803277e-01,  1.22202933e+00,
         9.75565434e-01, -8.42166245e-02,  1.31267536e+00,
        -9.60130692e-02,  6.88466311e-01, -7.44827926e-01,
         4.34886247e-01,  1.00935256e+00, -1.03147888e+00,
        -3.33178490e-02,  8.94104123e-01],
       [ 4.76273358e-01, -1.61304975e+00,  1.48866594e-01,
        -1.83206809e+00, -9.80634913e-02, -7.99689770e-01,
         3.63241166e-01,  7.21472740e-01,  1.60916185e+00,
         4.40313458e-01,  1.88291383e+00,  3.64577711e-01,
         8.68482232e-01,  9.65743184e-01, -1.76629674e+00,
         2.14440060e+00, -6.36306465e-01,  2.78741598e+00,
        -2.78

In [245]:
adenseaddt == a_added

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  

### Make Random

In [246]:
rand = SparseTensor.uniform(2,4)
rand

<SparseTensor <GPUBuffer with shape (8,)> with grad None>

In [247]:
rand.to_numpy()

array([[0.00222676, 0.00325276, 0.00873421, 0.0033025 ],
       [0.00599395, 0.00598574, 0.00821358, 0.0020109 ]])

In [248]:
rand.data

<GPUBuffer with shape (8,)>

### update vals

In [249]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)

In [250]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint col = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [251]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint topk = get_global_size(0);
      uint col = updateyidx[gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];m
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'GeForce GTX 1080 Ti' on 'NVIDIA CUDA' at 0x55c233cceeb0>:

<kernel>:17:32: error: use of undeclared identifier 'baseupdateidx'
        float val = updatevals[baseupdateidx+gid*topk+i];
                               ^
<kernel>:18:31: error: use of undeclared identifier 'baseidxidx'
        uint row = updatexidx[baseidxidx+i];
                              ^
<kernel>:32:49: error: use of undeclared identifier 'm'
                matData[idx2] = matData[idx2-1];m
                                                ^

(options: -I /home/fpaboim/.conda/envs/tinygrad/lib/python3.8/site-packages/pyopencl/cl)
(source saved as /tmp/tmpfnjucn88.cl)

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,1], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa), x_cp_buf, x_idx_buf, y_idx_buf)

resa = np.empty_like(adata)
resaidx = np.zeros(acols.shape).astype(np.uint32)
resannz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resa, adata_buf)
cl.enqueue_copy(queue, resaidx, acols_buf)
cl.enqueue_copy(queue, resannz, annzs_buf)

In [ ]:
resa.shape, resaidx.shape, resannz.shape, ellwa, a.T.shape

In [ ]:
adenseadd = to_dense(resa, resaidx, resannz, ellwa, a.shape)
adenseadd

In [ ]:
adenseadd - adense

In [ ]:
adenseadd == adense

In [ ]:
ellwa

In [ ]:
adata2 = adata.reshape(-1, ellwa)
adata2

In [ ]:
resa = resa.reshape(-1, ellwa)
resa

In [ ]:
resa - adata2

In [ ]:
acols

In [ ]:
resaidx

In [ ]:
resannz

In [ ]:
annz

### update vals2

In [ ]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)

In [ ]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint row = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint col = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,bs], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat), x_cp_buf, x_idx_buf, y_idx_buf)

resat = np.empty_like(adatat)
resaidxt = np.zeros(acolst.shape).astype(np.uint32)
resannzt = np.zeros(annzt.shape).astype(np.uint32)

cl.enqueue_copy(queue, resat, adatat_buf)
cl.enqueue_copy(queue, resaidxt, acolst_buf)
cl.enqueue_copy(queue, resannzt, annzst_buf)

In [ ]:
ellwa

In [ ]:
resat.shape, resaidxt.shape, resannzt.shape

In [ ]:
adenseaddt = to_dense(resat, resaidxt, resannzt, ellwat, a.T.shape)
adenseaddt

In [ ]:
adenseadd == adenseaddt.T

In [ ]:
adata2t = adatat.reshape(-1, ellwat)
adata2t

In [ ]:
resat = resat.reshape(-1, ellwat)
resat

In [ ]:
resat - adata2t

In [ ]:
acols

In [ ]:
resaidx

In [ ]:
resannz

In [ ]:
annz

# OTHER

import numpy as np
import pyopencl as cl

mf = cl.mem_flags

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

In [ ]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
val_out_buf = cl.Buffer(ctx, mf.READ_WRITE, 4*topk*topk)
x_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         uint topk,
                         __global  uint* xoutidx,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool larger = tempval > valx;
    bool larger2 = tempval2 > valy;
      
    posx += (larger)?1:0;
    posy += (larger2)?1:0;
  }
  //printf("posx:%i", posx);
  if (posx < topk) {
    xoutidx[posx] = gid;
  }
  if (posy < topk) {
    youtidx[posy] = gid;
  }
  if (gid < topk) {
    uint i = gid;
    for (uint j=0; j<topk; j++) {
      xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
    }
  }
}""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
event = knl(queue, [dim,], None, x_buf, y_buf, val_out_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

#event.wait()
val_out = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, val_out, val_out_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf, wait_for=[event])
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
val_out

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
asdf

In [ ]:
from __future__ import division

KERNEL_CODE = """
// Thread block size
#define BLOCK_SIZE %(block_size)d
// Matrix dimensions
// (chosen as multiples of the thread block size for simplicity)
#define WA %(w_a)d // Matrix A width
#define HA %(h_a)d // Matrix A height
#define WB %(w_b)d // Matrix B width
#define HB WA  // Matrix B height
#define WC WB  // Matrix C width
#define HC HA  // Matrix C height
/*
 * Copyright 1993-2009 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */
/* Matrix multiplication: C = A * B.
 * Device code.
 */
#define AS(j, i) As[i + j * BLOCK_SIZE]
#define BS(j, i) Bs[i + j * BLOCK_SIZE]
////////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! WA is A's width and WB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel __attribute__((reqd_work_group_size(16,16,1))) 
void
matrixMul( __global float* C, __global float* A, __global float* B)
{
    __local float As[BLOCK_SIZE*BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE*BLOCK_SIZE];
    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);
    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);
    // Index of the first sub-matrix of A processed by the block
    int aBegin = WA * BLOCK_SIZE * by;
    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + WA - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * WB;
    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;
    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {
        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + WA * ty + tx];
        BS(ty, tx) = B[b + WB * ty + tx];
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);
        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);
        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;
}
"""


In [ ]:
a2 = np.random.rand(4,4)

In [ ]:
a2

In [ ]:
a2.sum(axis=1)

In [ ]:
b2 = np.random.rand(4,4)

In [ ]:
b2

In [ ]:
b2.sum(axis=0)

In [ ]:
matmul = a2.dot(b2)
matmul